In [7]:
import pandas as pd
import json
from glob import glob
from types import SimpleNamespace
import seaborn as sns
import matplotlib.pyplot as plt
import periodictable as ptable
import numpy as np
from pathlib import Path

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.10f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
sns.set_context('talk')

In [8]:
dir_take1 = Path('').absolute().joinpath('calcs_zora_mw8_take1')
dir_take2 = Path('').absolute().joinpath('calcs_zora_mw8_take2')

atoms = ['Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe']

In [9]:
def MWEnergy(f):
    try:
        with open(f) as out:
            job = json.loads(out.read())
    except FileNotFoundError:
        return None
    try:
        if job['output']['success']:
            return float(job['output']['properties']['scf_energy']['E_tot'])
        else:
            return None
    except KeyError:
        return None

def numberOfSCFIterations(f):
    try:
        with open(f) as out:
            job = json.loads(out.read())
    except FileNotFoundError:
        return None
    try:
        if job['output']['success']:
            return int(len(job['output']['scf_calculation']['scf_solver']['cycles']))
        else:
            return None
    except KeyError:
        return None
    
def convData(f):
    try:
        with open(f) as out:
            job = json.loads(out.read())
    except FileNotFoundError:
        return None, None
    try:
        if job['output']['success']:
            update = job['output']['scf_calculation']['scf_solver']['cycles'][-1]['energy_update']
            residual = job['output']['scf_calculation']['scf_solver']['cycles'][-1]['mo_residual']
            return float(update), float(residual)
        else:
            return None, None
    except KeyError:
        return None, None
    
def getData(d, colname):
    """d: <pathlib.Path> to dir holding calcs"""
    table = []
    for atom in atoms:
        f_mw = d.joinpath(f'{atom}_pbe.json')
        e_mw = MWEnergy(f_mw)
        niter = numberOfSCFIterations(f_mw)
        update, residual = convData(f_mw)

        table.append((atom, niter, update, residual, e_mw))
    
    return pd.DataFrame(table, columns=['Atom', f'nIter', f'EnergyUpdate', f'MOResidual', f'Energy'])

In [16]:
t1 = getData(dir_take1, 'Take1')
t2 = getData(dir_take2, 'Take2')
df = t1.merge(t2, on='Atom', suffixes=('_Take1', '_Take2'))

df['Ratio_Take1'] = df.MOResidual_Take1 / df.EnergyUpdate_Take1
df['Ratio_Take2'] = df.MOResidual_Take2 / df.EnergyUpdate_Take2

df.EnergyUpdate_Take1 = df.EnergyUpdate_Take1.map(lambda x: f'{x:.1e}')
df.EnergyUpdate_Take2 = df.EnergyUpdate_Take2.map(lambda x: f'{x:.1e}')

df.MOResidual_Take1 = df.MOResidual_Take1.map(lambda x: f'{x:.1e}')
df.MOResidual_Take2 = df.MOResidual_Take2.map(lambda x: f'{x:.1e}')

df.Ratio_Take1 = (df.Ratio_Take1 * np.abs(df.Energy_Take1)).map(lambda x: f'{x:.1e}')
df.Ratio_Take2 = (df.Ratio_Take2 * np.abs(df.Energy_Take2)).map(lambda x: f'{x:.1e}')

df.nIter_Take1 = df.nIter_Take1.map(lambda x: f'{x:.0f}')
df.nIter_Take2 = df.nIter_Take2.map(lambda x: f'{x:.0f}')


cols = ['Atom', 'EnergyUpdate_Take1', 'EnergyUpdate_Take2', 'Energy_Take1', 'Energy_Take2', 'MOResidual_Take1', 'MOResidual_Take2']
print(df.reindex(sorted(df.columns), axis=1)[cols].to_latex(index=False))

\begin{tabular}{lllrrll}
\toprule
Atom & EnergyUpdate\_Take1 & EnergyUpdate\_Take2 &     Energy\_Take1 &     Energy\_Take2 & MOResidual\_Take1 & MOResidual\_Take2 \\
\midrule
  Rb &            1.8e-08 &            1.4e-04 & -3013.6351932639 & -3013.6346413261 &          1.6e-06 &          1.9e-06 \\
  Sr &            4.8e-08 &            1.4e-04 & -3215.7852970112 & -3215.7845703612 &          1.5e-06 &          1.7e-06 \\
   Y &            3.9e-06 &            1.7e-06 & -3425.7021986142 & -3425.7021668255 &          1.8e-06 &          8.9e-07 \\
  Zr &            1.6e-09 &                nan & -3643.6343461367 &              NaN &          9.0e-07 &              nan \\
  Nb &            1.2e-08 &            1.4e-04 & -3869.7494369446 & -3869.7489442815 &          1.1e-06 &          1.7e-06 \\
  Mo &            1.5e-08 &            8.6e-05 & -4104.1814757869 & -4104.1809948135 &          1.0e-06 &          9.8e-07 \\
  Tc &            8.3e-08 &            1.2e-04 & -4346.9529323659 & -